In [ ]:
%matplotlib inline

from utils.runner import RunsConfig, Runner
from utils.plotter import CanvasConfig
from traffic.environment import TrafficEnvironment
from traffic.agent import FixedCycleTrafficAgent, QLTrafficAgent, DQLTrafficAgent

canvas_config = CanvasConfig(
  [
    'system_total_stopped',
    'system_total_waiting_time',
    'system_mean_waiting_time',
    'system_mean_speed'
  ],
  1,
  100
)

traffic_env = TrafficEnvironment(
  net = '2wsi/2wsi.net.xml',
  rou = '2wsi/2wsi-2.rou.xml',
  seconds = 100000,
  delta_time = 10,
  yellow_time = 4,
  min_green = 10,
  max_green = 50
)

configs: list[RunsConfig] = [
  {
    'cls': FixedCycleTrafficAgent,
    'configs': [
      {
        'name': 'fixedcycle',
        'repeat': 1,
        'color': '#D9F029'
      }
    ]
  },
  {
    'cls': QLTrafficAgent,
    'configs': [
      {
        'name': 'ql_1',
        'repeat': 3,
        'color': '#EB7A00',
        'alpha': 0.1,
        'gamma': 0.75,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.9996
      },
      {
        'name': 'ql_2',
        'repeat': 3,
        'color': '#F5119A',
        'alpha': 0.1,
        'gamma': 0.9,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.9996
      }
    ]
  },
  {
    'cls': DQLTrafficAgent,
    'configs': [
      {
        'name': 'dql_1',
        'repeat': 3,
        'color': '#0012DB',
        'alpha': 0.1,
        'gamma': 0.75,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.99
      },
      {
        'name': 'dql_2',
        'repeat': 3,
        'color': '#16F5A4',
        'alpha': 0.1,
        'gamma': 0.9,
        'init_eps': 1,
        'min_eps': 0.01,
        'decay': 0.99
      }
    ]
  }
]

runner = Runner(canvas_config, traffic_env, configs)

runner.run(runner.learn(), 100000, True)